In [1]:
# Import the required libraries

import pandas as pd 
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re

!python -m spacy download en_core-web_sm

C:\Users\sorna\anaconda3\envs\20th-century-new\lib\site-packages\spacy\cli\info.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\sorna\anaconda3\envs\20th-century-new\lib\site-packages\spacy\cli\info.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core-web_sm/en_core-web_sm.tar.gz#egg=en_core==web_sm contains an egg fragment with a non-PEP 508 name. pip 25.3 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg

In [2]:
# Executed this command to solve the above error
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1-py3-none-any.whl

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 4.2 MB/s eta 0:00:03
     --- ------------------------------------ 1.0/12.8 MB 2.5 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.8 MB 2.5 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.8 MB 2.5 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/12.8 MB 1.3 MB/s eta 0:00:09
     ---- ----------------------------------- 1.3/12.8 MB 1.3 MB/s eta 0:00:09
     ---- ----------------------------------- 1.3/12.8 MB 1.3 MB/s eta 0:00:09
     ---- ----------------------------------- 1.3/12.8 MB 1.3 MB/s eta 0:00:09
     ---- ---------------------------------- 1.6/12.8 MB 814.6 kB/s eta 0:00:14
     ---- ---------------------------------- 1.6/12.8 MB 814.6 kB/s eta 0:00:14
     ----- --------------------------------- 1.8/12.8 MB 740.2 kB/s eta 0:00:15
     ----- --------------------------------- 1.8/12.8 MB

In [38]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

In [39]:
# Load the book

with open('events.txt', 'r', errors='ignore') as file: 
   data = file.read().replace( '\n', ' ')

book = NER(data)

In [40]:
displacy.render(book[273:20000], style= "ent", jupyter = True)

In [49]:
# Read the csv file with the list of countries

df = pd.read_csv("C:/Users/sorna/Desktop/Data Analysis/20th-Century/countries_list_20th_century_1.7.csv",index_col=False)
df['country_name'] = df['country_name'].str.strip()
countries = df['country_name'].tolist()

In [50]:
df

,country_name
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
206,South Ossetia
207,Taiwan
208,Transnistria
209,USSR


In [51]:
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 "China, People's Republic of",
 'Colombia',
 'Comoros',
 'Congo, Democratic Republic of the',
 'Congo, Republic of the',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia, The',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'G

In [52]:
# alias -> standard country name
country_aliases = {
    'USA': 'United States',
    'US': 'United States',
    'The United States': 'United States',
    'U.S.A' : 'United States',
    'U.S.' : 'United States',
    'America' : 'United States',
    'The Soviet Union' : 'USSR',
    'Soviet Union' : 'USSR',
    'Soviet' : 'USSR',
    'North Korea' : 'Korea,North',
    'South Korea' :'Korea,South',
    'Persia' : 'Iran',
    'Burma' : 'Myanmar',
    'North Vietnam' : 'Vietnam',
    'South Vietnam' : 'Vietnam',
    'Britain' : 'United Kingdom',
    'Great Britain' : 'United Kingdom',
    'North Africa' : 'North Africa',
}

In [53]:
country_aliases

{'USA': 'United States',
 'US': 'United States',
 'The United States': 'United States',
 'U.S.A': 'United States',
 'U.S.': 'United States',
 'America': 'United States',
 'The Soviet Union': 'USSR',
 'Soviet Union': 'USSR',
 'Soviet': 'USSR',
 'North Korea': 'Korea,North',
 'South Korea': 'Korea,South',
 'Persia': 'Iran',
 'Burma': 'Myanmar',
 'North Vietnam': 'Vietnam',
 'South Vietnam': 'Vietnam',
 'Britain': 'United Kingdom',
 'Great Britain': 'United Kingdom',
 'North Africa': 'North Africa'}

In [54]:
import re
from collections import Counter

# Load text
#with open("text.txt", "r", encoding="utf-8") as f:
 #   text = f.read()

with open('events.txt', 'r', encoding="utf-8") as file: 
   data = file.read().replace( '\n', ' ').lower()
# Prepare Counter
country_count = Counter()

# Step 3a: Count aliases
for alias, standard_name in country_aliases.items():
    matches = re.findall(r'\b' + re.escape(alias.lower()) + r'\b', data)
    country_count[standard_name] += len(matches)

# Step 3b: Count other countries (exclude those in alias values)
for country_name in countries:
    if country_name not in country_aliases.values():  # skip countries already counted via aliases
        matches = re.findall(r'\b' + re.escape(country_name.lower()) + r'\b', data)
        country_count[country_name] += len(matches)

print(country_count)


Counter({'USSR': 124, 'Germany': 41, 'United States': 39, 'Japan': 33, 'United Kingdom': 23, 'France': 17, 'Italy': 14, 'Poland': 13, 'India': 9, 'Russia': 9, 'Guinea': 8, 'Korea,North': 6, 'North Africa': 5, 'Austria': 5, 'Libya': 5, 'Norway': 5, 'Philippines': 5, 'Finland': 4, 'Greece': 4, 'Israel': 4, 'Pakistan': 4, 'Vietnam': 3, 'Cuba': 3, 'Denmark': 3, 'Hungary': 3, 'Lithuania': 3, 'Romania': 3, 'Korea,South': 2, 'Myanmar': 2, 'Albania': 2, 'Australia': 2, 'Bangladesh': 2, 'Belgium': 2, 'Cambodia': 2, 'Canada': 2, 'Egypt': 2, 'Estonia': 2, 'Latvia': 2, 'Netherlands': 2, 'Singapore': 2, 'Solomon Islands': 2, 'South Africa': 2, 'Ukraine': 2, 'Iran': 1, 'Afghanistan': 1, 'Algeria': 1, 'Angola': 1, 'Belarus': 1, 'Bulgaria': 1, 'Cape Verde': 1, 'Ghana': 1, 'Bissau': 1, 'Iraq': 1, 'Kenya': 1, 'Laos': 1, 'Lebanon': 1, 'Luxembourg': 1, 'Mexico': 1, 'Mongolia': 1, 'Morocco': 1, 'Mozambique': 1, 'Panama': 1, 'Papua New Guinea': 1, 'Rwanda': 1, 'São Tomé and Príncipe': 1, 'Seychelles': 1, 'S

In [55]:
df_count = pd.DataFrame(list(country_count.items()), columns=['Country', 'Count'])
#df_count.to_excel("country_count.xlsx", index=False)

In [56]:
df_count

,Country,Count
0,United States,39
1,USSR,124
2,"Korea,North",6
3,"Korea,South",2
4,Iran,1
...,...,...
207,Sahrawi Arab Democratic Republic,0
208,Somaliland,0
209,South Ossetia,0
210,Taiwan,0


In [57]:
df

,country_name
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
206,South Ossetia
207,Taiwan
208,Transnistria
209,USSR


In [58]:
# This code is used to get the list of the entities present in each sentence.

df_sentences = []

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
       entity_list = [ent.text for ent in sent.ents]
       df_sentences.append({"sentence": sent, "entities": entity_list})

df_sentences = pd.DataFrame(df_sentences)

In [59]:
df_sentences

,sentence,entities
0,( ),[]
1,"(Key, events, of, the, 20th, century, -, Wikip...","[the 20th century, Navigation \t Main, Contr..."
2,"(Search, , Search, ...",[]
3,"(account, , Log, in, , Personal, tool...","[Log in Personal, the 20th century, th..."
4,"("", The, war, to, end, all, wars, "", :, World,...","[World War I, Spanish, 1.3]"
...,...,...
1576,"(This, page, was, last, edited, on, 22, Septem...","[22 September 2025, 07:30Â, UTC]"
1577,"(Text, is, available, under, the, Creative, Co...",[the Creative Commons Attribution-ShareAlike 4...
1578,"(By, using, this, site, ,, you, agree, to, the...",[the Terms of Use and Privacy Policy]
1579,"(WikipediaÂ, ®, is, a, registered, trademark, ...","[WikipediaÂ®, the Wikimedia Foundation, Inc.]"


In [62]:
# Function to filter out entities not of interest

def filter_entity(ent_list, df):
       return [ent for ent in ent_list
                  if ent in countries]

In [63]:
df_sentences['country_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, df))

In [66]:
df_sentences.head(39)

,sentence,entities,country_entities
0,( ),[],[]
1,"(Key, events, of, the, 20th, century, -, Wikip...","[the 20th century, Navigation \t Main, Contr...",[]
2,"(Search, , Search, ...",[],[]
3,"(account, , Log, in, , Personal, tool...","[Log in Personal, the 20th century, th...",[]
4,"("", The, war, to, end, all, wars, "", :, World,...","[World War I, Spanish, 1.3]",[]
5,"(1.3.1, Economic, depression, , 1.3.2)",[],[]
6,"(The, rise, of, dictatorship, , 1.4,...","[1.4 Global war:, World War II, 1.4.1]",[]
7,"(The, war, in, Europe, , 1.4.2, Blitzk...","[Europe, 1.4.2]",[]
8,"(Operation, Overlord, , 1.4.6, Final, ...",[],[]
9,"(Allied, offensive, , 1.4.10, Final, d...",[Allied],[]


In [67]:
# Filter out sentences that don’t have any character entities

df_sentences_filtered = df_sentences[df_sentences['country_entities'].map(len) > 0]

df_sentences_filtered.tail(10)

,sentence,entities,country_entities
1098,"("", The, forgotten, violence, that, helped, In...",[India],[India]
1102,"("", Indian, Independence, Day, :, everything, ...","[Indian Independence Day, India, Pakistan, 70 ...","[India, Pakistan]"
1110,"("", The, Philippines, ,, 1898â€“1946, |, US, H...","[Philippines, 1898â€“1946, US, Art & Archives]",[Philippines]
1138,"("", Colonial, Cartographies, ,, Postcolonial, ...","[Colonial Cartographies, Enduring Failures of ...",[Afghanistan]
1174,"(The, Moldovans, :, Romania, ,, Russia, ,, and...","[Moldovans, Romania, Russia, the Politics of C...","[Romania, Russia]"
1230,"("", Selling, ', Operation, Passage, to, Freedo...","[Thomas Dooley, the Religious Overtones of Ear...",[Vietnam]
1259,"("", Stuck, in, Endless, Preliminaries, :, Viet...","[Vietnam, the Battle of the Paris Peace Table,...",[Vietnam]
1507,"("", Anti, -, American, Behavior, in, the, Midd...","[Anti-American, the Middle East, a Field Exper...",[Lebanon]
1512,"(The, Rise, of, China, and, India, :, A, New, ...","[The Rise of China, India]",[India]
1513,"(Singapore, :, World, Scientific, .)",[Singapore],[Singapore]


In [68]:
# Defining relationships

# window size = 5 : this defines how many sentences will be looked at simultaneously
relationships = [] # create an empty list

for i in range(df_sentences_filtered.index[-1]):
     end_i = min(i+5, df_sentences_filtered.index[-1])
     country_list = sum((df_sentences_filtered.loc[i: end_i].country_entities), [])

     # Remove duplicated characters that are next to each other
     country_unique = [
         country_list[i] for i in range(len(country_list))
         if (i==0) or country_list[i] != country_list[i-1]
     ]

     if len(country_unique) > 1:
          for idx, a in enumerate( country_unique[:-1]):
              b = country_unique[idx + 1]
              relationships.append({"source": a, "target": b})

In [69]:
relationship_df = pd.DataFrame(relationships)

relationship_df

,source,target
0,France,Russia
1,France,Russia
2,Russia,Germany
3,Germany,Russia
4,France,Russia
...,...,...
660,India,Singapore
661,India,Singapore
662,India,Singapore
663,India,Singapore


In [70]:
# Sort the cases with a- >b and b- >a
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df.head(5)

,source,target
0,France,Russia
1,France,Russia
2,Germany,Russia
3,Germany,Russia
4,France,Russia


In [71]:
relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()

In [72]:
relationship_df

,source,target,value
0,France,Russia,11
1,Germany,Russia,21
2,Germany,Italy,21
3,Austria,Germany,11
4,Germany,Spain,1
...,...,...,...
90,Greece,Italy,2
91,Italy,Japan,6
92,Romania,Russia,6
93,India,Lebanon,1


In [73]:
relationship_df.to_csv('C:/Users/sorna/Desktop/Data Analysis/20th-Century/Relationship_count.csv', index= False)